In [6]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

In [7]:
#data
df=pd.read_csv('C:\Kate\Python\Authorship Attribution\data\AllTweets.csv')
author1='NASA'
author2='RichardDawkins'
df_kk=df.loc[(df['author'] == author1)]
df_hc=df.loc[(df['author'] == author2)]
df=df_kk.append(df_hc,ignore_index=True)
len(df)

21749

In [8]:
import random
#2000 random sample rows for KK
rows = random.sample(df_kk.index, 2000)
df_kk = df_kk.ix[rows]
#2000 random sample rows for HC
rows = random.sample(df_hc.index, 2000)
df_hc = df_hc.ix[rows]
#join back together
df=df_kk.append(df_hc,ignore_index=True)
len(df)

4000

In [9]:
#data pre-processing
df.drop(df[df.retweet==True].index, inplace=True)
df['num_of_words'] = df["text"].str.split().apply(len)
df.drop(df[df.num_of_words<4].index, inplace=True)
df["text"].replace(r"http\S+", "URL", regex=True,inplace=True)
df["text"].replace(r"@\S+", "REF", regex=True ,inplace=True)
df["text"].replace(r"(\d{1,2})[/.-](\d{1,2})[/.-](\d{2,4})+", "DATE", regex=True,inplace=True)
df["text"].replace(r"(\d{1,2})[/:](\d{2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
df["text"].replace(r"(\d{1,2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
df["text"].replace(r"\d+", "NUM", regex=True,inplace=True)
len(df)

3993

In [10]:
#http://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
pos_code_map={'CC':'A','CD':'B','DT':'C','EX':'D','FW':'E','IN':'F','JJ':'G','JJR':'H','JJS':'I','LS':'J','MD':'K','NN':'L','NNS':'M',
'NNP':'N','NNPS':'O','PDT':'P','POS':'Q','PRP':'R','PRP$':'S','RB':'T','RBR':'U','RBS':'V','RP':'W','SYM':'X','TO':'Y','UH':'Z',
'VB':'1','VBD':'2','VBG':'3','VBN':'4','VBP':'5','VBZ':'6','WDT':'7','WP':'8','WP$':'9','WRB':'@'}
code_pos_map={v: k for k, v in pos_code_map.iteritems()}
#Python 3 inv_map = {v: k for k, v in my_map.items()}

In [11]:
def convert(tag):
    try:
        code=pos_code_map[tag]
    except:
        code='?'
    return code
def inv_convert(code):
    try:
        tag=code_pos_map[code]
    except:
        tag='?'
    return tag

In [12]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag, word_tokenize
def pos_tags(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    return "".join(convert(tag) for (word, tag) in nltk.pos_tag(text_processed))
def text_pos_inv_convert(text):
    return "-".join(inv_convert(c.upper()) for c in text)

In [13]:
df['text_pos']=df.apply(lambda x: pos_tags(x['text']), axis=1)

In [14]:
df.ix[:,['author','text','text_pos']].head()

,author,text,text_pos
0,NASA,What's up with Earth's changing ice sheets? Ch...,82WFN53LM1WCGGLNNNN
1,NASA,The #stsNUM crew will check out the flight & r...,CLLK1WCLLLMA1FLYNFNNNNN
2,NASA,RS-NUM ENGINE TEST UNDERWAY NOW: Watch live on...,NNNNNNN5FNLN
3,NASA,A massive cluster of galaxies located about NU...,CGLFM4FNNBGMTNN6MF3N
4,NASA,I'm getting real-time search results at TweetG...,R53GLLMFNN


In [15]:
df_features=pd.DataFrame()

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
for a in df.author.unique():
    v = CountVectorizer(analyzer='char',ngram_range=(7, 7))
    ngrams = v.fit_transform(df[df['author'] == a]['text_pos'])
    df_t=pd.DataFrame(
    {'Feature': v.get_feature_names(),
     'Count': list(ngrams.sum(axis=0).flat),
     'Author': a
    })
    #
    df_features=df_features.append(df_t,ignore_index=True)

In [17]:
df_features['Feature_POS']=df_features.apply(lambda x: text_pos_inv_convert(x['Feature']), axis=1)

In [18]:
df_features[~df_features.Feature.isin(df_features[df_features['Author'] != author2].Feature)].sort_values('Count', ascending=False).ix[:,['Author','Count','Feature','Feature_POS']].head()

,Author,Count,Feature,Feature_POS
40379,RichardDawkins,33,nnnnnng,NNP-NNP-NNP-NNP-NNP-NNP-JJ
34022,RichardDawkins,18,lfnnnng,NN-IN-NNP-NNP-NNP-NNP-JJ
39651,RichardDawkins,14,nnfnngl,NNP-NNP-IN-NNP-NNP-JJ-NN
38311,RichardDawkins,10,nglgnll,NNP-JJ-NN-JJ-NNP-NN-NN
39718,RichardDawkins,9,nnglgnl,NNP-NNP-JJ-NN-JJ-NNP-NN


In [19]:
df_features[~df_features.Feature.isin(df_features[df_features['Author'] != author1].Feature)].sort_values('Count', ascending=False).ix[:,['Author','Count','Feature','Feature_POS']].head()

,Author,Count,Feature,Feature_POS
10199,NASA,17,lfcnnnf,NN-IN-DT-NNP-NNP-NNP-IN
5391,NASA,9,cnnnnnl,DT-NNP-NNP-NNP-NNP-NNP-NN
2734,NASA,9,64y1fnn,VBZ-VBN-TO-VB-IN-NNP-NNP
6986,NASA,9,fnlalll,IN-NNP-NN-CC-NN-NN-NN
2048,NASA,7,4y1fnnn,VBN-TO-VB-IN-NNP-NNP-NNP


In [20]:
from sklearn.cross_validation import train_test_split
twt_train, twt_test, author_train, author_test = train_test_split(df.ix[:,['text','text_pos']], df['author'], test_size=0.4, random_state=42)

In [21]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
def text_process(text):
    """
    Takes in a string of text, then performs the following:
    1. Tokenizes and removes punctuation
    3. Stems
    4. Returns a list of the cleaned text
    """

    # tokenizing
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    
    
    # steming
    porter_stemmer = PorterStemmer()
    
    text_processed = [porter_stemmer.stem(word) for word in text_processed]
    

    return text_processed

In [22]:
ScoreSummaryByModelParams = list()

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

In [24]:
from sklearn.base import BaseEstimator, TransformerMixin
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

In [25]:
class TextAndTextCodedExtractor(BaseEstimator, TransformerMixin):
    """Extract the text & text_pos from a tweet in a single pass.
    """
    def fit(self, x, y=None):
        return self

    def transform(self, tweets):
        features=tweets.ix[:,['text_pos','text']].to_records(index=False)

        return features

In [26]:
def ModelParamsEvaluation (f_union,model,params,comment):
    pipeline = Pipeline([
    # Extract the text & text_coded
    ('textandtextcoded', TextAndTextCodedExtractor()),

    # Use FeatureUnion to combine the features from text and text_coded
    ('union', f_union, ),

    # Use a  classifier on the combined features
    ('clf', model),
    ])
    grid_search = GridSearchCV(estimator=pipeline, param_grid=params, verbose=1, cv=10)
    grid_search.fit(twt_train, author_train)
    #best score
    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(params.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        ScoreSummaryByModelParams.append([comment,grid_search.best_score_,"\t%s: %r" % (param_name, best_parameters[param_name])])    
 

In [27]:
f1_union=FeatureUnion(
        transformer_list=[
              # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char')),
            ])),               

        ],
    )

In [28]:
from sklearn.naive_bayes import BernoulliNB
p = {
    'union__char__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__char__tfidf__ngram_range': ((2, 2), (3, 3)), 
    'union__char__tfidf__max_features': (None, 5000, 10000, 50000),
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char')

Fitting 10 folds for each of 144 candidates, totalling 1440 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    9.5s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:   38.6s
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:  1.4min
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:  2.6min
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed:  4.1min
[Parallel(n_jobs=1)]: Done 1440 out of 1440 | elapsed:  4.7min finished


Best score: 0.959
Best parameters set:
	clf__alpha: 0.1
	union__char__tfidf__max_df: 0.5
	union__char__tfidf__max_features: None
	union__char__tfidf__ngram_range: (3, 3)


In [29]:
f1_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling word features from the text
            ('word', Pipeline([
            ('selector', ItemSelector(key='text')),
            ('tfidf',    TfidfVectorizer(analyzer='word')),
            ])),              

        ],
    )

In [30]:
p = {
    'union__word__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__word__tfidf__ngram_range': ((1, 1),(2, 2), (3, 3),(4,4),(5,5)), 
    'union__word__tfidf__max_features': (None, 5000, 10000, 50000),
    'union__word__tfidf__stop_words': (None, 'english'),
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, word')

Fitting 10 folds for each of 720 candidates, totalling 7200 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    7.1s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:   30.4s
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:  1.1min
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:  2.0min
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed:  3.2min
[Parallel(n_jobs=1)]: Done 1799 tasks       | elapsed:  4.6min
[Parallel(n_jobs=1)]: Done 2449 tasks       | elapsed:  6.3min
[Parallel(n_jobs=1)]: Done 3199 tasks       | elapsed:  8.2min
[Parallel(n_jobs=1)]: Done 4049 tasks       | elapsed: 10.4min
[Parallel(n_jobs=1)]: Done 4999 tasks       | elapsed: 12.8min
[Parallel(n_jobs=1)]: Done 6049 tasks       | elapsed: 15.5min
[Parallel(n_jobs=1)]: Done 7199 tasks       | elapsed: 18.5min
[Parallel(n_jobs=1)]: Done 7200 out of 7200 | elapsed: 18.5min finished


Best score: 0.965
Best parameters set:
	clf__alpha: 0.1
	union__word__tfidf__max_df: 0.75
	union__word__tfidf__max_features: None
	union__word__tfidf__ngram_range: (1, 1)
	union__word__tfidf__stop_words: None


In [31]:
f1_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling word features from the text
            ('text', Pipeline([
            ('selector', ItemSelector(key='text')),
            ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process)),
            ])),              

        ],
    )

In [32]:
p = {
    'union__text__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__text__tfidf__ngram_range': ((1, 1),(2, 2), (3, 3),(4,4),(5,5)), 
    'union__text__tfidf__max_features': (None, 5000, 10000, 50000),
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, stemmed words, no stop words')

Fitting 10 folds for each of 360 candidates, totalling 3600 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   25.8s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:  1.7min
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:  3.9min
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:  7.0min
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed: 10.9min
[Parallel(n_jobs=1)]: Done 1799 tasks       | elapsed: 15.8min
[Parallel(n_jobs=1)]: Done 2449 tasks       | elapsed: 21.4min
[Parallel(n_jobs=1)]: Done 3199 tasks       | elapsed: 28.0min
[Parallel(n_jobs=1)]: Done 3600 out of 3600 | elapsed: 31.5min finished


Best score: 0.969
Best parameters set:
	clf__alpha: 0.1
	union__text__tfidf__max_df: 0.75
	union__text__tfidf__max_features: 5000
	union__text__tfidf__ngram_range: (1, 1)


In [33]:
f1_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling pos tag features  from the text_pos
            ('text_pos', Pipeline([
            ('selector', ItemSelector(key='text_pos')),
            ('tfidf',    TfidfVectorizer(analyzer='char')),
            ])),                  

        ],
    )

In [34]:
p = {
    'union__text_pos__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__text_pos__tfidf__ngram_range': ((3, 3), (4, 4),(5,5),(6,6),(7,7)), 
    'union__text_pos__tfidf__max_features': (None, 5000, 10000, 50000),
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, POS tags')

Fitting 10 folds for each of 360 candidates, totalling 3600 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    5.4s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:   21.1s
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:   47.4s
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:  1.4min
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed:  2.2min
[Parallel(n_jobs=1)]: Done 1799 tasks       | elapsed:  3.2min
[Parallel(n_jobs=1)]: Done 2449 tasks       | elapsed:  4.3min
[Parallel(n_jobs=1)]: Done 3199 tasks       | elapsed:  5.7min


Best score: 0.731
Best parameters set:
	clf__alpha: 1
	union__text_pos__tfidf__max_df: 0.5
	union__text_pos__tfidf__max_features: None
	union__text_pos__tfidf__ngram_range: (3, 3)


[Parallel(n_jobs=1)]: Done 3600 out of 3600 | elapsed:  6.4min finished


In [35]:
df_ScoreSummaryByModelParams=DataFrame(ScoreSummaryByModelParams,columns=['Method','BestScore','BestParameter'])
df_ScoreSummaryByModelParams.sort_values(['BestScore'],ascending=False,inplace=True)
df_ScoreSummaryByModelParams

,Method,BestScore,BestParameter
9,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.969102,\tclf__alpha: 0.1
12,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.969102,"\tunion__text__tfidf__ngram_range: (1, 1)"
11,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.969102,\tunion__text__tfidf__max_features: 5000
10,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.969102,\tunion__text__tfidf__max_df: 0.75
8,"Bernoulli Naive Bayes, word",0.965344,\tunion__word__tfidf__stop_words: None
4,"Bernoulli Naive Bayes, word",0.965344,\tclf__alpha: 0.1
5,"Bernoulli Naive Bayes, word",0.965344,\tunion__word__tfidf__max_df: 0.75
6,"Bernoulli Naive Bayes, word",0.965344,\tunion__word__tfidf__max_features: None
7,"Bernoulli Naive Bayes, word",0.965344,"\tunion__word__tfidf__ngram_range: (1, 1)"
0,"Bernoulli Naive Bayes, char",0.958664,\tclf__alpha: 0.1


In [36]:
f2_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
            # Pipeline for pulling word features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',stop_words=None,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),        

        ],

    )

In [37]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f2_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + word')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   15.4s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   18.8s finished


Best score: 0.969
Best parameters set:
	clf__alpha: 0.001


In [38]:
f2_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
            # Pipeline for pulling stememd word features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),        

        ],

    )

In [39]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f2_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + stemmed word')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   32.2s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   39.4s finished


Best score: 0.971
Best parameters set:
	clf__alpha: 0.01


In [40]:
f3_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
            # Pipeline for pulling word features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',stop_words=None,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),    
            # Pipeline for pulling word features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),        

        ],

    )

In [41]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f3_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + word + stemmed word')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   36.0s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   44.3s finished


Best score: 0.970
Best parameters set:
	clf__alpha: 0.01


In [42]:
f3_union=FeatureUnion(
        transformer_list=[
             # Pipeline for pulling word stemmed features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded with POS tags
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [43]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f3_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + stemmed word + POS tags')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   34.5s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   42.3s finished


Best score: 0.966
Best parameters set:
	clf__alpha: 0.01


In [44]:
f3_union=FeatureUnion(
        transformer_list=[
             # Pipeline for pulling word stemmed features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded with POS tags
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [45]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f3_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + word + POS tags')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   17.7s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   21.9s finished


Best score: 0.966
Best parameters set:
	clf__alpha: 0.001


In [46]:
f4_union=FeatureUnion(
        transformer_list=[

            # Pipeline for pulling word features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',stop_words=None,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
             # Pipeline for pulling word features after word_processing from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [47]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f4_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + word + stemmed word + POS tag')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   38.7s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   47.5s finished


Best score: 0.970
Best parameters set:
	clf__alpha: 0.01


In [48]:
df_ScoreSummaryByModelParams=DataFrame(ScoreSummaryByModelParams,columns=['Method','BestScore','BestParameter'])
df_ScoreSummaryByModelParams.sort_values(['BestScore'],ascending=False,inplace=True)
df_ScoreSummaryByModelParams

,Method,BestScore,BestParameter
18,"Bernoulli Naive Bayes, char + stemmed word",0.971190,\tclf__alpha: 0.01
22,"Bernoulli Naive Bayes, char + word + stemmed w...",0.970355,\tclf__alpha: 0.01
19,"Bernoulli Naive Bayes, char + word + stemmed word",0.970355,\tclf__alpha: 0.01
9,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.969102,\tclf__alpha: 0.1
17,"Bernoulli Naive Bayes, char + word",0.969102,\tclf__alpha: 0.001
12,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.969102,"\tunion__text__tfidf__ngram_range: (1, 1)"
10,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.969102,\tunion__text__tfidf__max_df: 0.75
11,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.969102,\tunion__text__tfidf__max_features: 5000
20,"Bernoulli Naive Bayes, char + stemmed word + P...",0.965762,\tclf__alpha: 0.01
21,"Bernoulli Naive Bayes, char + word + POS tags",0.965762,\tclf__alpha: 0.001


In [68]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc,precision_score, accuracy_score, recall_score, f1_score
from scipy import interp

In [69]:
ScoreSummaryByVector = list()

In [70]:
def PredictionEvaluation(author_test_b,author_predictions_b,comment):
    Precision=precision_score(author_test_b,author_predictions_b)
    print ('Precision: %0.3f' % (Precision))
    Accuracy=accuracy_score(author_test_b,author_predictions_b)
    print ('Accuracy: %0.3f' % (Accuracy))
    Recall=recall_score(author_test_b,author_predictions_b)
    print ('Recall: %0.3f' % (Recall))
    F1=f1_score(author_test_b,author_predictions_b)
    print ('F1: %0.3f' % (F1))
    print ('Confussion matrix:')
    print (confusion_matrix(author_test_b,author_predictions_b))
    ROC_AUC=roc_auc_score(author_test_b,author_predictions_b)
    print ('ROC-AUC: %0.3f' % (ROC_AUC))
    ScoreSummaryByVector.append([Precision,Accuracy,Recall,F1,ROC_AUC,comment])

In [71]:
def ModelRun (f_union,model):
    pipeline = Pipeline([
    # Extract the text & text_coded
    ('textandtextcoded', TextAndTextCodedExtractor()),

    # Use FeatureUnion to combine the features from text and text_coded
    ('union', f_union, ),

    # Use a  classifier on the combined features
    ('clf', model),
    ])
    pipeline.fit(twt_train, author_train)
    author_predicted = pipeline.predict(twt_test)
    
    feature_names=list()
    for p in (pipeline.get_params()['union'].transformer_list):
        fn=(p[0],pipeline.get_params()['union'].get_params()[p[0]].get_params()['tfidf'].get_feature_names())
        feature_names.append(fn)
    df_fn=pd.DataFrame()
    for fn in feature_names:
        df_fn= df_fn.append(pd.DataFrame(
        {'FeatureType': fn[0],
         'Feature': fn[1]
        }),
        ignore_index=True)    
    
    from sklearn.preprocessing import LabelBinarizer
    lb = LabelBinarizer()
    author_test_b = lb.fit_transform(author_test.values)
    author_predicted_b  = lb.fit_transform(author_predicted)
    return (df_fn,pipeline.get_params()['clf'],author_predicted,author_predicted_b, author_test_b)

In [72]:
def most_informative_feature_for_binary_classification(feature_names, classifier):
    class_labels = classifier.classes_

    topnvalues_class0 = sorted(zip(classifier.coef_[0], feature_names['Feature'].values, feature_names['FeatureType'].values))
    topnvalues_class1 = sorted(zip(classifier.coef_[0], feature_names['Feature'].values, feature_names['FeatureType'].values), reverse=True)

    topn_df_class0=pd.DataFrame(topnvalues_class0, columns=['Coef','Feature','FeatureType'])
    topn_df_class0['Author']=class_labels[0]
    
    topn_df_class1=pd.DataFrame(topnvalues_class1, columns=['Coef','Feature','FeatureType'])
    topn_df_class1['Author']=class_labels[1]    
    
    topn_df=topn_df_class0.append(topn_df_class1)
    
        
    return topn_df

In [73]:
f2_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),
            # Pipeline for pulling stememd word features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.75,max_features=5000)),
            ])),        

        ],

    )

In [74]:
(feature_names,clf,author_predicted,author_predicted_b, author_test_b)=ModelRun(f2_union,BernoulliNB(alpha=0.01))

In [75]:
PredictionEvaluation(author_predicted_b, author_test_b,'char+stemmed word')

Precision: 0.981
Accuracy: 0.979
Recall: 0.976
F1: 0.978
Confussion matrix:
[[800  15]
 [ 19 764]]
ROC-AUC: 0.979


In [76]:
f3_union=FeatureUnion(
        transformer_list=[
             # Pipeline for pulling word stemmed features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.75,max_features=5000)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded with POS tags
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [77]:
(feature_names,clf,author_predicted,author_predicted_b, author_test_b)=ModelRun(f3_union,BernoulliNB(alpha=0.01))

In [78]:
PredictionEvaluation(author_predicted_b, author_test_b,'char+stemmed word+POS tag')

Precision: 0.979
Accuracy: 0.974
Recall: 0.968
F1: 0.974
Confussion matrix:
[[794  16]
 [ 25 763]]
ROC-AUC: 0.974


In [79]:
f4_union=FeatureUnion(
        transformer_list=[

            # Pipeline for pulling word features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',stop_words=None,ngram_range=(1, 1),max_df=0.75,max_features=None)),
            ])),
                    
             # Pipeline for pulling word features after word_processing from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [80]:
(feature_names,clf,author_predicted,author_predicted_b, author_test_b)=ModelRun(f4_union,BernoulliNB(alpha=0.01))

In [81]:
PredictionEvaluation(author_predicted_b, author_test_b,'char+word+stemmed word+POS tag')

Precision: 0.982
Accuracy: 0.973
Recall: 0.963
F1: 0.973
Confussion matrix:
[[790  14]
 [ 29 765]]
ROC-AUC: 0.973


In [82]:
df_ScoreSummaryByVector=DataFrame(ScoreSummaryByVector,columns=['Precision','Accuracy','Recall','F1','ROC-AUC','Vector'])
df_ScoreSummaryByVector.sort_values(['F1'],ascending=False,inplace=True)
df_ScoreSummaryByVector

,Precision,Accuracy,Recall,F1,ROC-AUC,Vector
0,0.980745,0.978723,0.975734,0.978233,0.978665,char+stemmed word
1,0.979461,0.974343,0.968274,0.973835,0.974261,char+stemmed word+POS tag
2,0.982028,0.973091,0.963476,0.972664,0.973032,char+word+stemmed word+POS tag


In [83]:
TopFeatures_df=most_informative_feature_for_binary_classification(feature_names, clf)

In [84]:
df1=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='char')),['Author','Coef','Feature']].head(10)
df1.rename(columns={'Coef':'CoefChar','Feature':'Char'}, inplace=True)
df1.reset_index(inplace=True)
df2=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='word')),['Coef','Feature']].head(10)
df2.rename(columns={'Coef':'CoefWord','Feature':'Word'}, inplace=True)
df2.reset_index(inplace=True)
df3=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='text')),['Coef','Feature']].head(10)
df3.rename(columns={'Coef':'CoefText','Feature':'Text'}, inplace=True)
df3.reset_index(inplace=True)
df4=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='text_pos')),['Coef','Feature']].head(10)
df4.rename(columns={'Coef':'CoefTextPOS','Feature':'TextPOS'}, inplace=True)
df4['TextPOS']=df4.apply(lambda x: text_pos_inv_convert(x['TextPOS']), axis=1)
df4.reset_index(inplace=True)
df_kk_top_features = pd.concat([df1,df2,df3,df4],axis=1)
df_kk_top_features.drop('index', axis=1, inplace=True)
df_kk_top_features

,Author,CoefChar,Char,CoefWord,Word,CoefText,Text,CoefTextPOS,TextPOS
0,RichardDawkins,-0.786156,he,-0.701405,url,-0.845652,the,-1.111849,NNP-NNP-NNP
1,RichardDawkins,-0.898761,ing,-0.845652,the,-1.055943,to,-1.382994,IN-NNP-NNP
2,RichardDawkins,-0.923301,to,-1.055943,to,-1.163141,of,-1.799984,NNP-IN-NNP
3,RichardDawkins,-1.025673,re,-1.163141,of,-1.251107,a,-1.845791,DT-JJ-NN
4,RichardDawkins,-1.030271,to,-1.402861,is,-1.402861,is,-1.866843,NN-IN-NNP
5,RichardDawkins,-1.039530,ng,-1.597966,in,-1.585845,i,-1.944226,NNP-JJ-NN
6,RichardDawkins,-1.089599,of,-1.626833,by,-1.589869,s,-1.949989,NNP-NNP-IN
7,RichardDawkins,-1.096961,on,-1.678344,ref,-1.597966,in,-1.991290,NNP-NNP-JJ
8,RichardDawkins,-1.163141,of,-1.687194,it,-1.626833,by,-1.991290,DT-NN-IN
9,RichardDawkins,-1.173723,is,-1.840597,for,-1.647975,it,-2.009527,IN-DT-NN


In [85]:
df1=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='char')),['Author','Coef','Feature']].head(10)
df1.rename(columns={'Coef':'CoefChar','Feature':'Char'}, inplace=True)
df1.reset_index(inplace=True)
df2=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='word')),['Coef','Feature']].head(10)
df2.rename(columns={'Coef':'CoefWord','Feature':'Word'}, inplace=True)
df2.reset_index(inplace=True)
df3=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='text')),['Coef','Feature']].head(10)
df3.rename(columns={'Coef':'CoefText','Feature':'Text'}, inplace=True)
df3.reset_index(inplace=True)
df4=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='text_pos')),['Coef','Feature']].head(10)
df4.rename(columns={'Coef':'CoefTextPOS','Feature':'TextPOS'}, inplace=True)
df4['TextPOS']=df4.apply(lambda x: text_pos_inv_convert(x['TextPOS']), axis=1)
df4.reset_index(inplace=True)
df_kk_top_features = pd.concat([df1,df2,df3,df4],axis=1)
df_kk_top_features.drop('index', axis=1, inplace=True)
df_kk_top_features

,Author,CoefChar,Char,CoefWord,Word,CoefText,Text,CoefTextPOS,TextPOS
0,NASA,-11.708509,#d,-11.708509,aasnum,-11.708509,aasnum,-11.708509,VB-VBG-RP
1,NASA,-11.708509,#e,-11.708509,aasnumpic,-11.708509,aasnump,-11.708509,VB-WP-DT
2,NASA,-11.708509,#g,-11.708509,abcs,-11.708509,abel,-11.708509,VB-WRB-DT
3,NASA,-11.708509,#j,-11.708509,abell,-11.708509,abinumghzd,-11.708509,VB-WRB-JJ
4,NASA,-11.708509,#m,-11.708509,abinumghzd,-11.708509,aboard,-11.708509,VB-WRB-NNS
5,NASA,-11.708509,#v,-11.708509,aboard,-11.708509,abscicon,-11.708509,VB-WRB-PRP
6,NASA,-11.708509,#y,-11.708509,abscicon,-11.708509,abund,-11.708509,VB-CC-DT
7,NASA,-11.708509,a.,-11.708509,abundant,-11.708509,academia,-11.708509,VB-CC-EX
8,NASA,-11.708509,aq,-11.708509,academia,-11.708509,access,-11.708509,VB-CC-TO
9,NASA,-11.708509,cy,-11.708509,accepting,-11.708509,accredit,-11.708509,VB-CD-NNS


In [86]:
author_predicted=pd.DataFrame(author_predicted,columns=['predicted'])
df_wrong_result = pd.concat([twt_test.reset_index(),author_test.reset_index(),author_predicted], axis=1)
df_wrong_result.drop('index', axis=1, inplace=True)
df_wrong_result.drop('text_pos', axis=1, inplace=True)
df_wrong_result=df_wrong_result[df_wrong_result['author']<>df_wrong_result['predicted']]
df_wrong_result.head(10)

,text,author,predicted
36,.REF We know you've been 'Across the Universe'...,NASA,RichardDawkins
43,NASA logos: The Meatball v. The Worm. Which do...,NASA,RichardDawkins
140,"REF Glad you like it! Actually, we have severa...",NASA,RichardDawkins
155,Comments from REF about Saturday's tragic shoo...,NASA,RichardDawkins
233,RT REF You have to love REF they do have a sen...,NASA,RichardDawkins
243,Good news! REF received a green beacon from th...,NASA,RichardDawkins
253,REF Both places are fascinating to study and h...,NASA,RichardDawkins
268,People didn't always think airplanes could car...,NASA,RichardDawkins
349,My on-stage conversation with the very interes...,RichardDawkins,NASA
390,Using binoculars to project an image on the wa...,RichardDawkins,NASA
